In [ ]:
import json
from datetime import datetime
from typing import Dict, List

import numpy as np
import seaborn as sns
import tensorflow as tf
from matplotlib import pyplot as plt
from sklearn.metrics import (accuracy_score, confusion_matrix, f1_score,
                             precision_score, recall_score)
from tensorflow import keras
from termcolor import colored

from src.models.geometric_figure import GeometricFigure
from src.services.geometric_figure import (get_geometric_figures,
                                           get_train_test_validation_split)
from src.services.result import save_model_results
from src.utils.files import create_directory_if_not_exists

In [ ]:
IMAGE_SIZE = (128, 128)
TEST_RATIO = 0.2
VALIDATION_RATIO = 0.1
DATA_VERSIONS = ['2023-04-24']
MODEL_RESULTS_PATH = 'data/results.json'
NUMBER_OF_REPETITIONS = 5
EPOCHS = 100

In [ ]:
ai_models: Dict[str, keras.Model] = {
    'P1': lambda: keras.Sequential([
        keras.layers.InputLayer(input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 1)),
        keras.layers.Flatten(),
        keras.layers.Dense(128, activation='relu'),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(32, activation='relu'),
        keras.layers.Dense(6, activation='softmax')
    ]),
    'P2': lambda: keras.Sequential([
        keras.layers.InputLayer(input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 1)),
        keras.layers.Flatten(),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(32, activation='relu'),
        keras.layers.Dense(6, activation='softmax')
    ]),
    'CNN1': lambda: keras.Sequential([
        keras.layers.InputLayer(input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 1)),
        keras.layers.Conv2D(32, (3, 3), activation='relu'),
        keras.layers.MaxPooling2D((4, 4)),
        keras.layers.Conv2D(32, (3, 3), activation='relu'),
        keras.layers.MaxPooling2D((4, 4)),
        keras.layers.Flatten(),
        keras.layers.Dense(8, activation='relu'),
        keras.layers.Dense(6, activation='softmax')
    ]),
    'CNN2': lambda: keras.Sequential([
        keras.layers.InputLayer(input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 1)),
        keras.layers.Conv2D(32, (3, 3), activation='relu'),
        keras.layers.MaxPooling2D((4, 4)),
        keras.layers.Conv2D(64, (3, 3), activation='relu'),
        keras.layers.MaxPooling2D((4, 4)),
        keras.layers.Flatten(),
        keras.layers.Dense(8, activation='relu'),
        keras.layers.Dense(6, activation='softmax')
    ]),
    # 'CNN3': lambda: keras.Sequential([
    #     keras.layers.InputLayer(input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 1)),
    #     keras.layers.Conv2D(64, (3, 3), activation='relu'),
    #     keras.layers.Conv2D(64, (3, 3), activation='relu'),
    #     keras.layers.MaxPooling2D((2, 2)),
    #     keras.layers.Dropout(0.25),
    #     keras.layers.Conv2D(128, (3, 3), activation='relu'),
    #     keras.layers.Conv2D(128, (3, 3), activation='relu'),
    #     keras.layers.MaxPooling2D((2, 2)),
    #     keras.layers.Dropout(0.25),
    #     keras.layers.Flatten(),
    #     keras.layers.Dense(256, activation='relu'),
    #     keras.layers.Dropout(0.5),
    #     keras.layers.Dense(6, activation='softmax')
    # ])
}

In [ ]:
geometric_figures_data_versions_dict: Dict[str, List[GeometricFigure]] = {
    version: get_geometric_figures(f'data/{version}', IMAGE_SIZE) for version in DATA_VERSIONS
}
for version, geometric_figures in geometric_figures_data_versions_dict.items():
    print(f'Loaded {colored(len(geometric_figures), "green")} geometric figures for version {colored(version, "green")}')

In [ ]:
def save_metrics(y_test: np.ndarray, y_test_predicted: np.ndarray, model_name: str, version: str, datetime_now: str):
    accuracy = accuracy_score(y_test, y_test_predicted)
    precision = precision_score(y_test, y_test_predicted, average='weighted')
    recall = recall_score(y_test, y_test_predicted, average='weighted')
    f1 = f1_score(y_test, y_test_predicted, average='weighted')

    print(f'Acurácia: {accuracy:.2%}')
    print(f'Precisão: {precision:.2%}')
    print(f'Sensibilidade: {recall:.2%}')
    print(f'F1: {f1:.2%}')

    metrics_dict = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }
    path_metrics = f'data/metrics/{version}/{model_name}/{datetime_now}/metrics.json'
    create_directory_if_not_exists(path_metrics)
    with open(path_metrics, 'w') as file:
        content = json.dumps(metrics_dict, indent=4)
        file.write(content)

    confusion = confusion_matrix(y_test, y_test_predicted)
    labels = ['circle', 'square', 'triangle', 'failed circle', 'failed square', 'failed triangle']
    fig = plt.figure(figsize=(8, 6))
    sns.heatmap(confusion, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
    plt.title(f'Matriz de confusão - {version}')
    plt.xlabel('Classe Prevista')
    plt.ylabel('Classe Verdadeira')
    path = f'data/metrics/{version}/{model_name}/{datetime_now}/confusion_matrix.svg'
    create_directory_if_not_exists(path)
    fig.savefig(path, bbox_inches='tight', dpi=300)
    plt.show()

In [ ]:
for data_version in DATA_VERSIONS:
    geometric_figures = geometric_figures_data_versions_dict[data_version]
    for i in range(NUMBER_OF_REPETITIONS):
        x_train, y_train, x_test, y_test, x_validation, y_validation = get_train_test_validation_split(
            geometric_figures,
            test_ratio=TEST_RATIO,
            validation_ratio=VALIDATION_RATIO,
            shuffle=True
        )
        # data_generator = keras.preprocessing.image.ImageDataGenerator(
        #     rotation_range=360,
        #     width_shift_range=0.1,
        #     height_shift_range=0.1,
        #     horizontal_flip=True,
        #     vertical_flip=True,
        # )
        # train_generator = data_generator.flow(x_train, y_train)
        for name, create_model in ai_models.items():
            model = create_model()
            model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
            print(f'Training {name}')
            # model.fit(train_generator, epochs=EPOCHS, validation_data=(x_validation, y_validation))
            model.fit(x_train, y_train, epochs=EPOCHS, validation_data=(x_validation, y_validation))
            print(f'Evaluating {name}')
            loss, accuracy = model.evaluate(x_test, y_test)
            print(f'Saving {name}')
            datetime_now = datetime.now().strftime('%Y-%m-%d %H-%M-%S')
            model.save(f'data/models/{data_version}/{name}/{datetime_now}.h5')
            save_model_results(name, accuracy, datetime_now, data_version, MODEL_RESULTS_PATH)
            predictions = model.predict(x_test)
            y_test_predicted = tf.argmax(predictions, axis=1).numpy()
            save_metrics(y_test, y_test_predicted, name, data_version, datetime_now)
